In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/mARVIN/Downloads/input_data_train.csv')

In [3]:
data.head()

,location,product,date,sa_quantity,temp_mean,temp_max,temp_min,sunshine_quant,event,price
0,1193,1,2011-12-31,5,10.635,11.19,10.08,56.0,New Year's Eve,1.48
1,1193,1,2012-01-01,0,8.845,10.76,6.93,213.0,New Year's Day,NaN
2,1193,1,2012-01-02,2,5.510,7.24,3.78,285.0,New Year's Day observed,1.50
3,1193,1,2012-01-03,2,8.915,11.61,6.22,205.0,2nd January (substitute day),1.50
4,1193,1,2012-01-04,0,7.380,8.87,5.89,224.0,NaN,NaN


In [4]:
if(data['temp_mean'].isnull().values.any()):
    data['temp_mean'] = data['temp_mean'].fillna(value=data['temp_mean'].mean())
    
if(data['temp_min'].isnull().values.any()):
    data['temp_min'] = data['temp_min'].fillna(value=data['temp_min'].mean())
    
if(data['temp_max'].isnull().values.any()):
    data['temp_max'] = data['temp_max'].fillna(value=data['temp_max'].mean())
    
if(data['sunshine_quant'].isnull().values.any()):
    data['sunshine_quant'] = data['sunshine_quant'].fillna(value=data['sunshine_quant'].mean())
    
if(data['event'].isnull().values.any()):
    data['event'] = data['event'].fillna(value='None')

if(data['price'].isnull().values.any()):
    data['price'] = data['price'].fillna(value=0)

In [5]:
print(data['temp_mean'].isnull().values.any())
print(data['temp_min'].isnull().values.any())
print(data['temp_max'].isnull().values.any())
print(data['sunshine_quant'].isnull().values.any())
print(data['event'].isnull().values.any())
print(data['price'].isnull().values.any())

False
False
False
False
False
False


In [6]:
data[['Year','Month','Day']] = data.date.str.split(expand=True,pat='-')
data['Year'] = data['Year'].astype(float)
data['Month'] = data['Month'].astype(float)
data['Day'] = data['Day'].astype(float)

In [ ]:
data = data.drop(['temp_max','temp_min','event','date'],axis=1)

In [11]:
data.head()

,location,product,sa_quantity,temp_mean,sunshine_quant,price,Year,Month,Day
0,1193,1,5,10.635,56.0,1.48,2011.0,12.0,31.0
1,1193,1,0,8.845,213.0,0.00,2012.0,1.0,1.0
2,1193,1,2,5.510,285.0,1.50,2012.0,1.0,2.0
3,1193,1,2,8.915,205.0,1.50,2012.0,1.0,3.0
4,1193,1,0,7.380,224.0,0.00,2012.0,1.0,4.0


In [12]:
data.sort_values(by=['Year','Month','Day'])

,location,product,sa_quantity,temp_mean,sunshine_quant,price,Year,Month,Day
0,1193,1,5,10.635,56.0,1.480000,2011.0,12.0,31.0
548,1193,2,1,10.635,56.0,2.100000,2011.0,12.0,31.0
1096,1193,3,10,10.635,56.0,2.400000,2011.0,12.0,31.0
1644,1193,4,52,10.635,56.0,1.490385,2011.0,12.0,31.0
2192,1193,5,61,10.635,56.0,1.490164,2011.0,12.0,31.0
2740,1193,6,0,10.635,56.0,0.000000,2011.0,12.0,31.0
3288,1193,7,7,10.635,56.0,3.485714,2011.0,12.0,31.0
3836,1193,8,22,10.635,56.0,1.300000,2011.0,12.0,31.0
4384,1193,9,2,10.635,56.0,1.350000,2011.0,12.0,31.0
4932,1193,10,5,10.635,56.0,1.360000,2011.0,12.0,31.0


In [13]:
X = data.iloc[:,[0,1,3,4,5,6,7,8]].values
Y = data.iloc[:,2].values


In [14]:
Y

array([ 5,  0,  2, ..., 14, 14,  8], dtype=int64)

In [15]:
X

array([[1.1930e+03, 1.0000e+00, 1.0635e+01, ..., 2.0110e+03, 1.2000e+01,
        3.1000e+01],
       [1.1930e+03, 1.0000e+00, 8.8450e+00, ..., 2.0120e+03, 1.0000e+00,
        1.0000e+00],
       [1.1930e+03, 1.0000e+00, 5.5100e+00, ..., 2.0120e+03, 1.0000e+00,
        2.0000e+00],
       ...,
       [9.7000e+01, 3.6900e+02, 1.5620e+01, ..., 2.0130e+03, 6.0000e+00,
        2.8000e+01],
       [9.7000e+01, 3.6900e+02, 1.5870e+01, ..., 2.0130e+03, 6.0000e+00,
        2.9000e+01],
       [9.7000e+01, 3.6900e+02, 1.9235e+01, ..., 2.0130e+03, 6.0000e+00,
        3.0000e+01]])

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
regressor = RandomForestRegressor(n_estimators=80, random_state=0)
regressor.fit(X_train, Y_train)


In [17]:
regressor = pickle.load(open('RF.sav','rb'))


In [18]:
y_pred = regressor.predict(X_test)

In [19]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

Mean Absolute Error: 2.2094075771962105
Mean Squared Error: 30.468170621674307
Root Mean Squared Error: 5.519798059863631


In [77]:
import pickle
pickle.dump(regressor, open('RF.sav','wb'))